In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install unsloth
# Get latest Unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/genaiproj/AddressCorrector')

# Reading data

In [ ]:
import pandas as pd

# Load the Parquet file
df = pd.read_parquet("data/address_with_instructions.parquet")
df.head()


,OID_,AddNum_Pre,Add_Number,AddNum_Suf,AddNo_Full,St_PreMod,St_PreDir,St_PreTyp,St_PreSep,St_Name,...,SecondaryAddress,CityStateZip,FullAddress,FormattedFullAddress,task1_instruction,task1_groundtruth,task2_instruction,task2_groundtruth,noise_level,variant_idx
0,62241576,,1187.0,,1187.0,,north,,,pownal,...,,"pownal, vt, 5260","1187 north pownal road\n\npownal, vt, 5260","1187, north Pownal Road, Bennington County, Ve...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1187""...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1187""...",medium,0
1,19781905,,1124.0,,1124.0,,,,,judah bear,...,,"richmond, ky, 40475","1124 judah bear boulevard\n\nrichmond, ky, 40475","1124, Judah Bear Boulevard, Richmond, Madison ...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1124""...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1124""...",extreme,2
2,21790301,,27.0,,27.0,,,,,captain bellamy,...,,"ma, 2632","27 captain bellamy lane\n\nma, 2632","27, Captain Bellamy Lane, Centerville, Barnsta...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""27"",\...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""27"",\...",medium,0
3,29482843,,1676.0,,1676.0,,,,,englewood,...,,"mn, 55104-1113","1676 englewood avenue\n\nmn, 55104-1113","1676, Englewood Avenue, Ramsey County, Minneso...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1676""...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1676""...",extreme,2
4,74441520,,3820.0,,3820.0,,,,,duckhorn,...,"apt 2104, apt 2104","sacramento, ca, 95834-1363","3820 duckhorn drive\napt 2104, apt 2104\nsacra...","3820, Duckhorn Drive, Unit apt 2104, Sacrament...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""3820""...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""3820""...",medium,0


In [ ]:
# getting only mass data
df=df.loc[df['State']=='ma']
df.head()

,OID_,AddNum_Pre,Add_Number,AddNum_Suf,AddNo_Full,St_PreMod,St_PreDir,St_PreTyp,St_PreSep,St_Name,...,SecondaryAddress,CityStateZip,FullAddress,FormattedFullAddress,task1_instruction,task1_groundtruth,task2_instruction,task2_groundtruth,noise_level,variant_idx
2,21790301,,27.0,,27.0,,,,,captain bellamy,...,,"ma, 2632","27 captain bellamy lane\n\nma, 2632","27, Captain Bellamy Lane, Centerville, Barnsta...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""27"",\...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""27"",\...",medium,0
16,24211894,,42.0,,42.0,,,,,vadnais,...,,"ma, 1001","42 vadnais street\n\nma, 1001","42, Vadnais Street, Agawam, Hampden County, Ma...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""42"",\...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""42"",\...",medium,0
97,25195295,,28.0,,28.0,,,,,woolworth,...,,"ma, 1106","28 woolworth street\n\nma, 1106","28, Woolworth Street, Longmeadow, Hampden Coun...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""28"",\...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""28"",\...",medium,0
134,22248751,,75.0,,75.0,,,,,willow brook,...,,"ma, 1520","75 willow brook road\n\nma, 1520","75, Willow Brook Road, Holden, Worcester Count...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""75"",\...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""75"",\...",extreme,2
301,24346545,,1990.0,,1990.0,,,,,columbus,...,"137, 137","ma, 2119","1990 columbus avenue\n137, 137\nma, 2119","1990, Columbus Avenue, Unit 137, Roxbury, Suff...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1990""...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1990""...",extreme,2


In [ ]:
df.shape

(30000, 71)

In [ ]:
df['OID_'].nunique()

10000

In [ ]:
df['OID_'].iloc[:20000].nunique()

9639

In [ ]:
df['OID_'].iloc[20001:].nunique()

7004

In [ ]:
df.shape

(30000, 71)

Inference on new runtime

In [ ]:
# Load the fine tuned model and tokenizer
from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch

checkpoint_path = "models/llama3_sft_sfttrainer_MA/checkpoint-2500"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",  # base model
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
    device_map = "auto"
)

# Load LoRA adapter weights
model.load_adapter(checkpoint_path)


model.eval()


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=512, bias=False)
            (lora_dropout): ModuleDict(
       

In [ ]:
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template
import pandas as pd

# Patch tokenizer if not already done
tokenizer = get_chat_template(tokenizer, chat_template="llama-3")

# Start from rows after 20,000 (used for training)
df_infer = df.iloc[20000:].copy()

# 1. Start from the portion not used in training
df_infer = df.iloc[20000:].copy()

# 2. Task 1: Last 100 unique OIDs (clean base addresses)
df_task1 = df_infer.drop_duplicates("OID_").tail(100)

# 3. Task 2: Last 100 unique OIDs where task2_instruction exists (noisy/fixed addresses)
df_task2 = df_infer.drop_duplicates("OID_").tail(100)


results_task1 = []
results_task2 = []

# Inference for task1
for _, row in df_task1.iterrows():
    instruction = row["task1_instruction"]
    messages = [{"role": "user", "content": instruction}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(**inputs, max_new_tokens=512, temperature=0.7, top_p=0.9, do_sample=True)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    response = decoded.split("assistant")[-1].strip() if "assistant" in decoded else decoded

    results_task1.append({
        "OID_": row["OID_"],
        "instruction": instruction,
        "ground_truth": row["task1_groundtruth"],
        "model_output": response
    })

# Inference for task2
for _, row in df_task2.iterrows():
    instruction = row["task2_instruction"]
    messages = [{"role": "user", "content": instruction}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(**inputs, max_new_tokens=512, temperature=0.7, top_p=0.9, do_sample=True)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    response = decoded.split("assistant")[-1].strip() if "assistant" in decoded else decoded

    results_task2.append({
        "OID_": row["OID_"],
        "instruction": instruction,
        "ground_truth": row["task2_groundtruth"],
        "model_output": response
    })

#  Save both
pd.DataFrame(results_task1).to_csv("outputs/test_outputs/task1_inference_results.csv", index=False)
pd.DataFrame(results_task2).to_csv("outputs/test_outputs/task2_inference_results.csv", index=False)

print("✅ Saved task1 and task2 inference results separately!")


✅ Saved task1 and task2 inference results separately!


In [ ]:
# Task 1 Prompt for parsing address(single example)

from unsloth.chat_templates import get_chat_template

# Apply correct chat formatting for inference
tokenizer = get_chat_template(tokenizer, chat_template="llama-3")

messages = [
    {
        "role": "user",
        "content": '''Parse the following address into a structured JSON with these fields: AddNum_Pre, Add_Number, AddNum_Suf, St_PreDir, St_Name, St_PosTyp, St_PosDir, Building, Floor, Unit, Room, Uninc_Comm, Inc_Muni, County, State, Zip_Code.
Address: 13th Street 47 W 13th St, New York, NY 10011'''
    }
]

# Format prompt
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# Tokenize and generate
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

_ = model.generate(
    **inputs,
    streamer=streamer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

System: {
  "AddNum_Pre": "",
  "Add_Number": "13th",
  "AddNum_Suf": "47",
  "St_PreDir": "",
  "St_Name": "13th st",
  "St_PosTyp": "w 13th st",
  "St_PosDir": "",
  "Building": "",
  "Floor": "",
  "Unit": "",
  "Room": "",
  "Uninc_Comm": "",
  "Inc_Muni": "new york",
  "County": "nan",
  "State": "ny",
  "Zip_Code": "10011"
}


In [ ]:
# Task - 2 for rewriting the address(single example)

from unsloth.chat_templates import get_chat_template

# Apply correct chat formatting for inference
tokenizer = get_chat_template(tokenizer, chat_template="llama-3")

# Example: Instruction-only chat (no assistant content)
messages = [
    {"role": "user", "content": '''Fix the formatting, structure, correct any existing entities, or predict/add new values to the appropriate entities of this Address JSON. Expand common abbreviations (like st→street, ave→avenue), correct obvious errors (like leading zeros in numbers), generate new values to the appropriate entities, and standardize capitalization. Keep empty fields as empty strings. Do not return anything other than corrected Address JSON
Address JSON: {
  "AddNum_Pre": "",
  "Add_Number": "386",
  "AddNum_Suf": "",
  "St_PreDir": "",
  "St_Name": "barnaby",
  "St_PosTyp": "street",
  "St_PosDir": "southeast",
  "Building": "",
  "Floor": "",
  "Unit": "",
  "Room": "",
  "Uninc_Comm": "trvsnv3ngk9tj hwmueu7s",
  "Inc_Muni": "bdtgh6fgtl4n",
  "County": "qisprr8tt tog ptlpt6s3",
  "State": "dc",
  "Zip_Code": "20302"
}'''}
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True  # important for inference
)


In [ ]:
# generate output
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

_ = model.generate(
    **inputs,
    streamer=streamer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    use_cache=True
)


System: {
  "AddNum_Pre": "",
  "Add_Number": "3868",
  "AddNum_Suf": "",
  "St_PreDir": "",
  "St_Name": "barnaby",
  "St_PosTyp": "street",
  "St_PosDir": "southeast",
  "Building": "",
  "Floor": "",
  "Unit": "",
  "Room": "",
  "Uninc_Comm": "trinidad heights",
  "Inc_Muni": "washington",
  "County": "district of columbia",
  "State": "dc",
  "Zip_Code": "20032"
}


In [ ]:
# code